In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils.multiclass import unique_labels

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
#Import standard libraries

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
''' to learn more about itertools visit
    https://medium.com/@jasonrigden/a-guide-to-python-itertools-82e5a306cdf8'''
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
#Import keras functions

from keras import Sequential

'''Since we are using transfer learning let's import the model that we want to implement.Let's use VGG 19(19 layers) and Resnet-50 (50 layers of residual units). 
Residual units allow us to add more layers onto the model without a degradation in accuracy.
Let's try and compare the accuracy of the 2 models and see if the addtional layers do make a significant difference. '''

from keras.applications import VGG19,ResNet50

'Import the datagenerator to augment images'
from keras.preprocessing.image import ImageDataGenerator

'''Import the optimizers and leanring rate annealer (which will reduce the learning rate once a particular metric we choose(in this case validation error) 
does not reduce after a user defined number of epochs)'''
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau

'Lastly import the final layers that will be added on top of the base model'
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout

'Import to_categorical from the keras utils package to one hot encode the labels'
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
#Import dataset
from keras.datasets import cifar100

In [5]:
#Divide the data in Train, Validation and Test Datasets
'I had to turn the Internet setting to on to download load the dataset'
(x_train,y_train),(x_test,y_test)=cifar100.load_data()


In [6]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

In [7]:
#Print the dimensions of the datasets to make sure everything's kosher

print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))


In [8]:
#One hot encode the labels.Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10

y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)


In [9]:
# Lets print the dimensions one more time to see if things changed the way we expected

print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 100))
((15000, 32, 32, 3), (15000, 100))
((10000, 32, 32, 3), (10000, 100))


We can now begin the actual process of model building.I find that following a set process and following consistently makes learning this easier.So here is the process I follow:

*  Define the Data Augmentation (ImageDataGenerator) and Learning Rate Annealer (ReduceOnPlateau) functions
*  Build the model (Base Model + Flatten + Dense)
*  Check model summary
*  Initialize Batch Size,Number of Epochs
*  Compile model
*  Fit the model (We will use fit_generator since the data is fed to the model using an augmentation function
*  Evaluate the model on test data


In [10]:
#Data Augmentation Function: Let's define an instance of the ImageDataGenerator class and set the parameters.We have to instantiate for the Train,Validation and Test datasets
train_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1 )

val_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1)

test_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip= True,
                                    zoom_range=.1) 




In [11]:
#Fit the augmentation method to the data

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

It's not necessary to use a data generator for validation data while fitting the model, however I find that it gives better validation accuracy if it is used.This is probably because themodel can learn more generalized features if validation data is also augmented.
Ive used only a few of the many available functionalities of the augment function.To know more about the capabilities read this article:
https://towardsdatascience.com/image-augmentation-for-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085


In [12]:
'''Learning Rate Annealer: The learning rate can be modified after a set number of epochs or after a certain condition is met. We will use the latter and change the learning rate if 
the validation error does not reduce after a set number of epochs. To do this we will use the patience parameter.'''

lrr= ReduceLROnPlateau(
                       monitor='val_acc', #Metric to be measured
                       factor=.01, #Factor by which learning rate will be reduced
                       patience=3,  #No. of epochs after which if there is no improvement in the val_acc, the learning rate is reduced
                       min_lr=1e-5) #The minimum learning rate 

In [27]:
#Build the model

base_model_1 = ResNet50(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=y_train.shape[1])
base_model_2 = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [14]:
batch_size= 100
epochs=10

The next step is to define the learning rate for the optimizer we will use. I have chosen the SGD and Adam optimizer. The main difference between the 2 is that SGD uses the same learning rate for all parameters and updates all of them by the same amount. The learning rate does not change during training. Adam stands for Adaptive Moment estimation and maintains a separate learning rate for each parameter and updates them separately.

To understand this concept more please read this article:
https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

In [15]:
learn_rate=.001

sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


Since we're using a function to generate data, we have to use the argument fit_generator. Both the train data and the validation data will be generated using the augmentation methods we have previously defined. To use the fit_generator function we will define the following parameters:

generator.flow(x_train,y_train,batch_size)

Here we use generator.flow since the data is being generated from a numpy array. You could also have data available in folders in which case we would use flow_from_directory in which case the class names are inferred directly from the folder names within the train data folder

More information on this can be found by reading the official documentation:
https://keras.io/preprocessing/image/



Now that we have our code for the confusion matrix, let's make predictions on the test set and see how this model has performed

In [17]:
class_names = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 
                    'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 
                    'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'computer_keyboard', 
                    'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 
                    'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 
                    'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 
                    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 
                    'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 
                    'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']

To compare this with another model, let's try and use Resnet-50. Residual nets allow us to add deeper layers to the network, without having the problem of accuracy degradation. They do this by using skip connections, meaning they jump over layers.

To understand Resnets better please read the following links:

https://towardsdatascience.com/hitchhikers-guide-to-residual-networks-resnet-in-keras-385ec01ec8ff

https://towardsdatascience.com/residual-blocks-building-blocks-of-resnet-fd90ca15d6ec

## Model 1

resnet as base model, input shape = (32,32,3)

In [42]:
model_1=Sequential()
#Add the Dense layers along with activation and batch normalization
model_1.add(base_model_1)
# model_1.add(Flatten())
model_1.add(GlobalAveragePooling2D())
model_1.add(Dropout(.25))
model_1.add(Dense(256, activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dense(100, activation='softmax'))

In [43]:
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               524544    
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1024      
_________________________________________________________________
dense_15 (Dense)             (None, 100)               25700     
Total params: 24,138,980
Trainable params: 24,085,348
Non-trainable params: 53,632
___________________________________________________________

Immediately the first differences we see are that ResNet50 has a little more than 16 million extra parameters to train which is to be expected since it is a deeper model. Also, the number of units before the Flatten layer are 4 times that of the previous model.

In [44]:
#Compile the model 

model_1.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [45]:
import time
t0 = time.time()
model_1.fit_generator(train_generator.flow(x_train,y_train,batch_size=batch_size),
                     epochs=10,steps_per_epoch=x_train.shape[0]//batch_size,
                     validation_data=val_generator.flow(x_val,y_val,batch_size=batch_size),validation_steps=250,
                      verbose=1)
t1 = time.time()-t0

Epoch 1/10
350/350 [==============================] - 67s 193ms/step - loss: 4.7063 - acc: 0.0110 - val_loss: 4.6399 - val_acc: 0.0109
Epoch 2/10
350/350 [==============================] - 40s 115ms/step - loss: 4.5937 - acc: 0.0120 - val_loss: 4.5963 - val_acc: 0.0134
Epoch 3/10
350/350 [==============================] - 40s 114ms/step - loss: 4.5638 - acc: 0.0158 - val_loss: 4.5421 - val_acc: 0.0133
Epoch 4/10
350/350 [==============================] - 41s 116ms/step - loss: 4.4967 - acc: 0.0189 - val_loss: 5.0367 - val_acc: 0.0175
Epoch 5/10
350/350 [==============================] - 40s 115ms/step - loss: 4.4451 - acc: 0.0244 - val_loss: 7.0788 - val_acc: 0.0222
Epoch 6/10
350/350 [==============================] - 40s 114ms/step - loss: 4.3196 - acc: 0.0319 - val_loss: 4.3538 - val_acc: 0.0277
Epoch 7/10
350/350 [==============================] - 40s 114ms/step - loss: 4.2365 - acc: 0.0401 - val_loss: 4.2470 - val_acc: 0.0385
Epoch 8/10
350/350 [==============================] - 4

In [46]:
y_pred=model_1.predict_classes(x_test)
y_true=np.argmax(y_test,axis=1)

#Compute the confusion matrix
confusion_mtx=confusion_matrix(y_true,y_pred)

In [47]:
from sklearn.metrics import accuracy_score
print("Testing Accuracy: ", accuracy_score(y_true,y_pred))

Testing Accuracy:  0.0697


## Model 2
resnet as base model, input shape = (224,224,3)

In [29]:
from keras.layers import GlobalAveragePooling2D,UpSampling2D

model_2=Sequential()
#Add the Dense layers along with activation and batch normalization
model_2.add(UpSampling2D(size=(7, 7),interpolation='bilinear'))
model_2.add(base_model_2)
model_2.add(GlobalAveragePooling2D())
model_2.add(Dropout(.25))
model_2.add(Dense(256, activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dense(100, activation='softmax'))

In [30]:
model_2.build((None,32,32,3)) # `input_shape` is the shape of the input data
                         # e.g. input_shape = (None, 32, 32, 3)
model_2.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [31]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_4 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               524544    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dense_8 (Dense)              (None, 100)               25700     
Total para

In [24]:
#import cv2
#x_train_resized =  np.array([cv2.resize(img, (224, 224)) for img in x_train])
#x_val_resized =  np.array([cv2.resize(img, (224, 224)) for img in x_val])

In [32]:
import time
t0 = time.time()
model_2.fit_generator(train_generator.flow(x_train,y_train,batch_size=batch_size),
                     epochs=10,steps_per_epoch=x_train.shape[0]//batch_size,
                     validation_data=val_generator.flow(x_val,y_val,batch_size=batch_size),validation_steps=250,
                      verbose=1)
t2 = time.time()-t0

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
350/350 [==============================] - 274s 782ms/step - loss: 2.7114 - acc: 0.3162 - val_loss: 2.5922 - val_acc: 0.3481
Epoch 2/10
350/350 [==============================] - 251s 716ms/step - loss: 1.6625 - acc: 0.5361 - val_loss: 2.6908 - val_acc: 0.3308
Epoch 3/10
350/350 [==============================] - 251s 716ms/step - loss: 1.3069 - acc: 0.6240 - val_loss: 2.1857 - val_acc: 0.4344
Epoch 4/10
350/350 [==============================] - 251s 716ms/step - loss: 1.0809 - acc: 0.6804 - val_loss: 1.7159 - val_acc: 0.5405
Epoch 5/10
350/350 [==============================] - 251s 717ms/step - loss: 0.9120 - acc: 0.7265 - val_loss: 2.1205 - val_acc: 0.4847
Epoch 6/10
350/350 [==============================] - 251s 716ms/step - loss: 0.7845 - acc: 0.7601 - val_loss: 1.5458 - val_acc: 0.5858
Epoch 7/10
350/350 [==============================] - 250s 715ms/step - loss: 0.6631 - acc: 0.7955 - val_loss: 2.0495 - val_acc: 0.5251


In [34]:
y_pred=model_2.predict_classes(x_test)
y_true=np.argmax(y_test,axis=1)

#Compute the confusion matrix
confusion_mtx=confusion_matrix(y_true,y_pred)

In [35]:
print("Testing Accuracy: ", accuracy_score(y_true,y_pred))

Testing Accuracy:  0.6214


## Model 3
efficientnet as base model, input = (224,224,3)

In [58]:
#!pip install -U efficientnet

In [60]:
import efficientnet.keras as effnet
base_model_3 = effnet.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))

model_3 = Sequential()
model_3.add(UpSampling2D(size=(7, 7),interpolation='bilinear'))
model_3.add(base_model_3)
model_3.add(GlobalAveragePooling2D())
model_3.add(Dropout(.25))
model_3.add(Dense(256, activation='relu'))
model_3.add(BatchNormalization())
model_3.add(Dense(100, activation='softmax'))


In [61]:
model_3.build((None,32,32,3)) # `input_shape` is the shape of the input data
                         # e.g. input_shape = (None, 32, 32, 3)
model_3.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [64]:
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_6 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
efficientnet-b0 (Model)      (None, 7, 7, 1280)        4049564   
_________________________________________________________________
global_average_pooling2d_7 ( (None, 1280)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1280)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               327936    
_________________________________________________________________
batch_normalization_7 (Batch (None, 256)               1024      
_________________________________________________________________
dense_19 (Dense)             (None, 100)               25700     
Total para

In [62]:
t0 = time.time()
model_3.fit_generator(train_generator.flow(x_train,y_train,batch_size=batch_size),
                     epochs=10,steps_per_epoch=x_train.shape[0]//batch_size,
                     validation_data=val_generator.flow(x_val,y_val,batch_size=batch_size),validation_steps=250,
                      verbose=1)
t3 = time.time()-t0

Epoch 1/10
350/350 [==============================] - 308s 879ms/step - loss: 2.8905 - acc: 0.2805 - val_loss: 2.3785 - val_acc: 0.3785
Epoch 2/10
350/350 [==============================] - 271s 773ms/step - loss: 1.6806 - acc: 0.5193 - val_loss: 1.7612 - val_acc: 0.5158
Epoch 3/10
350/350 [==============================] - 270s 771ms/step - loss: 1.2747 - acc: 0.6289 - val_loss: 1.4293 - val_acc: 0.5951
Epoch 4/10
350/350 [==============================] - 269s 768ms/step - loss: 1.0227 - acc: 0.6908 - val_loss: 1.2560 - val_acc: 0.6464
Epoch 5/10
350/350 [==============================] - 269s 767ms/step - loss: 0.8426 - acc: 0.7417 - val_loss: 1.2065 - val_acc: 0.6601
Epoch 6/10
350/350 [==============================] - 270s 772ms/step - loss: 0.6972 - acc: 0.7834 - val_loss: 1.1419 - val_acc: 0.6815
Epoch 7/10
350/350 [==============================] - 270s 772ms/step - loss: 0.5898 - acc: 0.8144 - val_loss: 1.1815 - val_acc: 0.6775
Epoch 8/10
350/350 [============================

In [63]:
y_pred=model_3.predict_classes(x_test)
y_true=np.argmax(y_test,axis=1)
print("Testing Accuracy: ", accuracy_score(y_true,y_pred))

Testing Accuracy:  0.6899


## Model 4
Freeze

In [67]:
base_model_3.trainable = False
model_4 = Sequential()
model_4.add(UpSampling2D(size=(7, 7),interpolation='bilinear'))
model_4.add(base_model_3)
model_4.add(GlobalAveragePooling2D())
model_4.add(Dropout(.25))
model_4.add(Dense(256, activation='relu'))
model_4.add(BatchNormalization())
model_4.add(Dense(100, activation='softmax'))

In [68]:
model_4.build((None,32,32,3)) # `input_shape` is the shape of the input data
                         # e.g. input_shape = (None, 32, 32, 3)
model_4.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [70]:
model_4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_7 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
efficientnet-b0 (Model)      (None, 7, 7, 1280)        4049564   
_________________________________________________________________
global_average_pooling2d_8 ( (None, 1280)              0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 1280)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 256)               327936    
_________________________________________________________________
batch_normalization_8 (Batch (None, 256)               1024      
_________________________________________________________________
dense_21 (Dense)             (None, 100)               25700     
Total para

In [71]:
t0 = time.time()
model_4.fit_generator(train_generator.flow(x_train,y_train,batch_size=batch_size),
                     epochs=10,steps_per_epoch=x_train.shape[0]//batch_size,
                     validation_data=val_generator.flow(x_val,y_val,batch_size=batch_size),validation_steps=250,
                      verbose=1)
t4 = time.time()-t0

Epoch 1/10
350/350 [==============================] - 105s 300ms/step - loss: 0.5788 - acc: 0.8305 - val_loss: 1.2146 - val_acc: 0.6995
Epoch 2/10
350/350 [==============================] - 78s 222ms/step - loss: 0.3061 - acc: 0.9032 - val_loss: 1.2438 - val_acc: 0.7026
Epoch 3/10
350/350 [==============================] - 78s 222ms/step - loss: 0.2647 - acc: 0.9140 - val_loss: 1.2663 - val_acc: 0.7040
Epoch 4/10
350/350 [==============================] - 77s 221ms/step - loss: 0.2309 - acc: 0.9241 - val_loss: 1.2886 - val_acc: 0.7071
Epoch 5/10
350/350 [==============================] - 77s 221ms/step - loss: 0.2160 - acc: 0.9297 - val_loss: 1.3072 - val_acc: 0.7076
Epoch 6/10
350/350 [==============================] - 78s 223ms/step - loss: 0.2030 - acc: 0.9336 - val_loss: 1.3006 - val_acc: 0.7103
Epoch 7/10
350/350 [==============================] - 77s 221ms/step - loss: 0.1895 - acc: 0.9361 - val_loss: 1.3385 - val_acc: 0.7076
Epoch 8/10
350/350 [==============================] - 

In [72]:
y_pred=model_4.predict_classes(x_test)
y_true=np.argmax(y_test,axis=1)
print("Testing Accuracy: ", accuracy_score(y_true,y_pred))

Testing Accuracy:  0.7153


In [73]:
print(t1)
print(t2)
print(t3)
print(t4)

438.65637826919556
2538.4194946289062
2747.7093801498413
808.6735408306122
